In [1]:
import pandas as pd
import numpy as np
import os
import bar_chart_race as bcr
import ffmpeg
import matplotlib as mt

In [2]:
path = 'TagByDate2'
listDir = os.listdir(path)
tagBDF = pd.DataFrame()
for direc in listDir:
    df = pd.read_csv(os.path.join(path, direc), names=['date','tag','count'])
    tagBDF = pd.concat([tagBDF, df], axis=0)

In [3]:
tagBDF

,date,tag,count
0,2013-12-21,javascript,106
1,2016-06-14,c++,66
2,2010-05-20,mysql,6
3,2009-11-24,android,5
4,2011-02-09,css,37
...,...,...,...
532,2010-09-01,java,83
533,2011-10-08,mysql,21
534,2012-02-25,javascript,66
535,2014-02-13,android,148


In [4]:
tagBDF.sort_values(by=['date'], inplace=True)
tagBDF.reset_index(drop=True, inplace=True)
tagBDF

,date,tag,count
0,2008-08-01,sql,1
1,2008-08-01,asp.net,1
2,2008-08-01,c#,2
3,2008-08-02,c++,2
4,2008-08-02,c#,3
...,...,...,...
43390,2016-10-19,python,182
43391,2016-10-19,javascript,217
43392,2016-10-19,android,139
43393,2016-10-19,java,182


In [5]:
tagBDF["count"] = pd.to_numeric(tagBDF["count"])

In [6]:
tagBDF.to_csv('tagBDF.csv', index=False)

In [7]:
tagBDF_ = tagBDF.pivot_table(index=['date'], columns='tag', values='count', fill_value=0)
tagBDF_

tag,android,asp.net,c,c#,c++,css,html,ios,java,javascript,jquery,mysql,php,python,sql
date,,,,,,,,,,,,,,,
2008-08-01,0,1,0,2,0,0,0,0,0,0,0,0,0,0,1
2008-08-02,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0
2008-08-03,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
2008-08-04,0,3,0,14,0,0,0,0,0,0,0,0,0,0,0
2008-08-05,0,5,3,13,2,0,6,0,0,0,0,5,5,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-10-15,84,4,34,45,52,40,39,41,123,115,44,43,62,88,29
2016-10-16,60,15,29,59,28,29,45,40,101,105,29,29,62,107,32
2016-10-17,164,26,31,112,64,59,108,90,171,172,70,23,116,152,38


In [8]:
tagBDF_.index = pd.to_datetime(tagBDF_.index)
tagBDF_ = tagBDF_.resample('M').sum() #Agrupa por mes
tagBDF_

tag,android,asp.net,c,c#,c++,css,html,ios,java,javascript,jquery,mysql,php,python,sql
date,,,,,,,,,,,,,,,
2008-08-31,2,112,68,233,96,48,42,0,112,70,7,32,66,40,106
2008-09-30,5,441,241,870,550,185,239,0,670,321,91,137,276,288,318
2008-10-31,0,400,233,848,420,112,197,5,468,354,116,109,269,285,263
2008-11-30,0,331,124,613,323,94,156,3,413,212,84,102,235,191,193
2008-12-31,4,336,124,697,406,139,164,2,371,261,86,137,218,204,207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30,3440,740,593,3377,1546,1876,2732,1872,4083,5902,2558,1476,3420,3433,1496
2016-07-31,3760,587,637,3035,1493,1863,2792,1783,3864,5919,2556,1478,3402,3365,1275
2016-08-31,3572,705,612,3235,1376,1990,2720,1819,4114,6024,2525,1441,3297,3259,1231


In [9]:
for c in tagBDF_.columns:
    tagBDF_[c] = tagBDF_[c].cumsum()
tagBDF_

tag,android,asp.net,c,c#,c++,css,html,ios,java,javascript,jquery,mysql,php,python,sql
date,,,,,,,,,,,,,,,
2008-08-31,2,112,68,233,96,48,42,0,112,70,7,32,66,40,106
2008-09-30,7,553,309,1103,646,233,281,0,782,391,98,169,342,328,424
2008-10-31,7,953,542,1951,1066,345,478,5,1250,745,214,278,611,613,687
2008-11-30,7,1284,666,2564,1389,439,634,8,1663,957,298,380,846,804,880
2008-12-31,11,1620,790,3261,1795,578,798,10,2034,1218,384,517,1064,1008,1087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30,206718,77403,70617,270385,134618,115074,157916,107557,301258,308508,206863,109445,255011,157509,99670
2016-07-31,210478,77990,71254,273420,136111,116937,160708,109340,305122,314427,209419,110923,258413,160874,100945
2016-08-31,214050,78695,71866,276655,137487,118927,163428,111159,309236,320451,211944,112364,261710,164133,102176


In [10]:
import seaborn as sns
from matplotlib.colors import ListedColormap
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
my_cmap = ListedColormap(sns.color_palette("Paired"))

In [11]:
bcr.bar_chart_race(df=tagBDF_, 
                   filename="tagRaceF3.mp4", 
                   n_bars=10, 
                   title='Tags from 10% of Stack Overflow Q&A by frequency', 
                   period_fmt='%B %Y',
                   filter_column_colors=True,
                   cmap="Light24")#,
                   #period_length=50)

'''bcr.bar_chart_race(df=qtByDate_, filename=None,
    orientation='h',
    sort='desc',
    n_bars=6,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=10,
    interpolate_period=False,
    label_bars=True,
    bar_size=.95,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='%B %d, %Y',
    period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                      's': f'Total deaths: {v.nlargest(6).sum():,.0f}',
                                      'ha': 'right', 'size': 8, 'family': 'Courier New'},
    perpendicular_bar_func='median',
    period_length=500,
    figsize=(5, 3),
    dpi=144,
    cmap='dark12',
    title='COVID-19 Deaths by Country',
    title_size='',
    bar_label_size=7,
    tick_label_size=7,
    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=False)'''

c:\python39\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
c:\python39\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))


"bcr.bar_chart_race(df=qtByDate_, filename=None,\n    orientation='h',\n    sort='desc',\n    n_bars=6,\n    fixed_order=False,\n    fixed_max=True,\n    steps_per_period=10,\n    interpolate_period=False,\n    label_bars=True,\n    bar_size=.95,\n    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},\n    period_fmt='%B %d, %Y',\n    period_summary_func=lambda v, r: {'x': .99, 'y': .18,\n                                      's': f'Total deaths: {v.nlargest(6).sum():,.0f}',\n                                      'ha': 'right', 'size': 8, 'family': 'Courier New'},\n    perpendicular_bar_func='median',\n    period_length=500,\n    figsize=(5, 3),\n    dpi=144,\n    cmap='dark12',\n    title='COVID-19 Deaths by Country',\n    title_size='',\n    bar_label_size=7,\n    tick_label_size=7,\n    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},\n    scale='linear',\n    writer=None,\n    fig=None,\n    bar_kwargs={'alpha': .7},\n    filter_column_colors=False)"